# SH3GL2 - Single gene analysis in GP2 data

## Description

Using "individual level data"

### 0. Getting Started

- Loading Python libraries
- Defining functions
- Installing packages

### 1. Copy data from workspace to cloud environment

### 2. Extract SH3GL2

### 3. Annotate SH3GL2 variants

### 4. Extract coding/non-syn variants

### 5. Calculate frequency in cases versus controls

### 6. Calculate frequency (homozygotes) in cases versus controls

### 7. Save out results

## Getting Started

### Loading Python libraries

### Defining functions

In [1]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# Numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

In [2]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

### Set paths

In [3]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name: {WORKSPACE_NAME}')
print(f'Billing Project: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data: {WORKSPACE_BUCKET}')
print('')


## GP2 v5.0
## Explicitly define release v5.0 path 
GP2_RELEASE_PATH = 'gs://gp2tier2/release5_11052023'
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
print('GP2 v5.0')
print(f'Path to GP2 v5.0 Clinical Data: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v5.0 Raw Genotype Data: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v5.0 Imputed Genotype Data: {GP2_IMPUTED_GENO_PATH}')


## AMP-PD v3.0
## Explicitly define release v3.0 path 
AMP_RELEASE_PATH = 'gs://amp-pd-data/releases/2022_v3release_1115'
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'

AMP_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS'
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_PLINK_PFILES = os.path.join(AMP_WGS_RELEASE_PLINK_PATH, 'pfiles')

print('AMP-PD v3.0')
print(f'Path to AMP-PD v3.0 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PFILES}')
print('')

Billing and Workspace
Workspace Name: Endophilin-A
Billing Project: terra-18d8e41c
Workspace Bucket, where you can upload and download data: gs://fc-f7a400c1-827e-48f8-b7b6-90c488a000a4

GP2 v5.0
Path to GP2 v5.0 Clinical Data: gs://gp2tier2/release5_11052023/clinical_data
Path to GP2 v5.0 Raw Genotype Data: gs://gp2tier2/release5_11052023/raw_genotypes
Path to GP2 v5.0 Imputed Genotype Data: gs://gp2tier2/release5_11052023/imputed_genotypes
AMP-PD v3.0
Path to AMP-PD v3.0 Clinical Data: gs://amp-pd-data/releases/2022_v3release_1115/clinical
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink/pfiles



### Install packages

#### Install Plink 1.9 and Plink 2.0

In [4]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi


if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220603.zip
unzip -o plink2_linux_avx2_20220603.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink1.9 is already installed in /home/jupyter/tools/
Plink2 is already installed in /home/jupyter/tools/


In [5]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Remote restrictions

In [6]:
%%bash

# chmod plink 1.9 
chmod u+x /home/jupyter/tools/plink

In [7]:
%%bash

# chmod plink 2.0
chmod u+x /home/jupyter/tools/plink2

### Install ANNOVAR

In [8]:
%%bash

# Install ANNOVAR:
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

annovar is already installed in /home/jupyter/tools/


In [9]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Install ANNOVAR: Download sources of annotation 

In [10]:
%%bash

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20140902 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the 'humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.idx.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished d

In [11]:
%%bash
ls /home/jupyter/tools/annovar/

annotate_variation.pl
coding_change.pl
convert2annovar.pl
example
humandb
retrieve_seq_from_fasta.pl
table_annovar.pl
variants_reduction.pl


## Copy data from GP2 bucket to workspace

In [12]:
# Make a directory
print("Making a working directory")
WORK_DIR = f'/home/jupyter/SH3GL2_GP2/'
shell_do(f'mkdir -p {WORK_DIR}') # f' means f-string - contains expressions to execute the code

Making a working directory


Executing: mkdir -p /home/jupyter/SH3GL2_GP2/


In [13]:
# Check directory where GP2 data is
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_IMPUTED_GENO_PATH}')

Executing: gsutil -u terra-18d8e41c ls gs://gp2tier2/release5_11052023/imputed_genotypes


gs://gp2tier2/release5_11052023/imputed_genotypes/AAC/
gs://gp2tier2/release5_11052023/imputed_genotypes/AFR/
gs://gp2tier2/release5_11052023/imputed_genotypes/AJ/
gs://gp2tier2/release5_11052023/imputed_genotypes/AMR/
gs://gp2tier2/release5_11052023/imputed_genotypes/CAS/
gs://gp2tier2/release5_11052023/imputed_genotypes/EAS/
gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/
gs://gp2tier2/release5_11052023/imputed_genotypes/FIN/
gs://gp2tier2/release5_11052023/imputed_genotypes/MDE/
gs://gp2tier2/release5_11052023/imputed_genotypes/SAS/


In [14]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_IMPUTED_GENO_PATH}/FIN/chr9* {WORK_DIR}')

Executing: gsutil -u terra-18d8e41c -m cp -r gs://gp2tier2/release5_11052023/imputed_genotypes/FIN/chr9* /home/jupyter/SH3GL2_GP2/


Copying gs://gp2tier2/release5_11052023/imputed_genotypes/FIN/chr9_FIN_release5.psam...
Copying gs://gp2tier2/release5_11052023/imputed_genotypes/FIN/chr9_FIN_release5.log...
Copying gs://gp2tier2/release5_11052023/imputed_genotypes/FIN/chr9_FIN_release5.pvar...
Copying gs://gp2tier2/release5_11052023/imputed_genotypes/FIN/chr9_FIN_release5.pgen...
/ [4/4 files][  8.4 MiB/  8.4 MiB] 100% Done                                    
Operation completed over 4 objects/8.4 MiB.                                      


### Create a covariate file with GP2 data

In [15]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_CLINICAL_RELEASE_PATH}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_CLINICAL_RELEASE_PATH}/master_key_release5_final.csv {WORK_DIR}')

Executing: gsutil -u terra-18d8e41c ls gs://gp2tier2/release5_11052023/clinical_data


gs://gp2tier2/release5_11052023/clinical_data/master_key_release5_final.csv
gs://gp2tier2/release5_11052023/clinical_data/release5_data_dictionary.csv


Executing: gsutil -u terra-18d8e41c -m cp -r gs://gp2tier2/release5_11052023/clinical_data/master_key_release5_final.csv /home/jupyter/SH3GL2_GP2/


Copying gs://gp2tier2/release5_11052023/clinical_data/master_key_release5_final.csv...
/ [1/1 files][  2.5 MiB/  2.5 MiB] 100% Done                                    
Operation completed over 1 objects/2.5 MiB.                                      


In [16]:
cov = pd.read_csv(f'{WORK_DIR}/master_key_release5_final.csv')
cov.columns

Index(['GP2ID', 'GP2sampleID', 'manifest_id', 'phenotype', 'pheno_for_qc',
       'other_pheno', 'sex_for_qc', 'age', 'age_of_onset', 'age_at_diagnosis',
       'age_at_death', 'race_for_qc', 'family_history', 'region_for_qc',
       'study', 'pruned', 'pruned_reason', 'label', 'related'],
      dtype='object')

In [17]:
cov_reduced = cov[['GP2sampleID','GP2sampleID','sex_for_qc', 'age', 'phenotype']]
cov_reduced.columns = ['FID','IID', 'SEX','AGE','PHENO']
cov_reduced

,FID,IID,SEX,AGE,PHENO
0,IMMUNEPD_000001_s1,IMMUNEPD_000001_s1,1,61.0,PD
1,IMMUNEPD_000002_s1,IMMUNEPD_000002_s1,1,66.0,PD
2,IMMUNEPD_000003_s1,IMMUNEPD_000003_s1,2,55.0,Control
3,IMMUNEPD_000004_s1,IMMUNEPD_000004_s1,1,50.0,Control
4,IMMUNEPD_000005_s1,IMMUNEPD_000005_s1,2,74.0,Control
...,...,...,...,...,...
26723,MDGAP-EBB_000149_s1,MDGAP-EBB_000149_s1,1,NaN,Control
26724,MDGAP-EBB_000150_s1,MDGAP-EBB_000150_s1,2,NaN,Control
26725,MDGAP-EBB_000151_s1,MDGAP-EBB_000151_s1,1,NaN,Control
26726,MDGAP-EBB_000152_s1,MDGAP-EBB_000152_s1,2,NaN,Control


In [18]:
conditions = [
    (cov_reduced['PHENO'] == "PD"),
    (cov_reduced['PHENO'] == "Control")]

In [19]:
choices = [2,1]
cov_reduced['PHENOTYPE'] = np.select(conditions, choices, default=-9).astype(np.int64)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
cov_reduced

,FID,IID,SEX,AGE,PHENO,PHENOTYPE
0,IMMUNEPD_000001_s1,IMMUNEPD_000001_s1,1,61.0,PD,2
1,IMMUNEPD_000002_s1,IMMUNEPD_000002_s1,1,66.0,PD,2
2,IMMUNEPD_000003_s1,IMMUNEPD_000003_s1,2,55.0,Control,1
3,IMMUNEPD_000004_s1,IMMUNEPD_000004_s1,1,50.0,Control,1
4,IMMUNEPD_000005_s1,IMMUNEPD_000005_s1,2,74.0,Control,1
...,...,...,...,...,...,...
26723,MDGAP-EBB_000149_s1,MDGAP-EBB_000149_s1,1,NaN,Control,1
26724,MDGAP-EBB_000150_s1,MDGAP-EBB_000150_s1,2,NaN,Control,1
26725,MDGAP-EBB_000151_s1,MDGAP-EBB_000151_s1,1,NaN,Control,1
26726,MDGAP-EBB_000152_s1,MDGAP-EBB_000152_s1,2,NaN,Control,1


In [21]:
cov_reduced.reset_index(inplace=True)
cov_reduced.drop(columns=["index"], inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
cov_reduced.drop(columns=['PHENO'], inplace=True)

In [23]:
cov_reduced

,FID,IID,SEX,AGE,PHENOTYPE
0,IMMUNEPD_000001_s1,IMMUNEPD_000001_s1,1,61.0,2
1,IMMUNEPD_000002_s1,IMMUNEPD_000002_s1,1,66.0,2
2,IMMUNEPD_000003_s1,IMMUNEPD_000003_s1,2,55.0,1
3,IMMUNEPD_000004_s1,IMMUNEPD_000004_s1,1,50.0,1
4,IMMUNEPD_000005_s1,IMMUNEPD_000005_s1,2,74.0,1
...,...,...,...,...,...
26723,MDGAP-EBB_000149_s1,MDGAP-EBB_000149_s1,1,NaN,1
26724,MDGAP-EBB_000150_s1,MDGAP-EBB_000150_s1,2,NaN,1
26725,MDGAP-EBB_000151_s1,MDGAP-EBB_000151_s1,1,NaN,1
26726,MDGAP-EBB_000152_s1,MDGAP-EBB_000152_s1,2,NaN,1


In [24]:
sex = cov_reduced[['FID','IID','SEX']]
sex

,FID,IID,SEX
0,IMMUNEPD_000001_s1,IMMUNEPD_000001_s1,1
1,IMMUNEPD_000002_s1,IMMUNEPD_000002_s1,1
2,IMMUNEPD_000003_s1,IMMUNEPD_000003_s1,2
3,IMMUNEPD_000004_s1,IMMUNEPD_000004_s1,1
4,IMMUNEPD_000005_s1,IMMUNEPD_000005_s1,2
...,...,...,...
26723,MDGAP-EBB_000149_s1,MDGAP-EBB_000149_s1,1
26724,MDGAP-EBB_000150_s1,MDGAP-EBB_000150_s1,2
26725,MDGAP-EBB_000151_s1,MDGAP-EBB_000151_s1,1
26726,MDGAP-EBB_000152_s1,MDGAP-EBB_000152_s1,2


In [25]:
sex.to_csv(f'{WORK_DIR}/SEX.txt',index=False, sep="\t")

In [26]:
pheno = cov_reduced[['FID','IID','PHENOTYPE']]
pheno

,FID,IID,PHENOTYPE
0,IMMUNEPD_000001_s1,IMMUNEPD_000001_s1,2
1,IMMUNEPD_000002_s1,IMMUNEPD_000002_s1,2
2,IMMUNEPD_000003_s1,IMMUNEPD_000003_s1,1
3,IMMUNEPD_000004_s1,IMMUNEPD_000004_s1,1
4,IMMUNEPD_000005_s1,IMMUNEPD_000005_s1,1
...,...,...,...
26723,MDGAP-EBB_000149_s1,MDGAP-EBB_000149_s1,1
26724,MDGAP-EBB_000150_s1,MDGAP-EBB_000150_s1,1
26725,MDGAP-EBB_000151_s1,MDGAP-EBB_000151_s1,1
26726,MDGAP-EBB_000152_s1,MDGAP-EBB_000152_s1,1


In [27]:
pheno.to_csv(f'{WORK_DIR}/PHENO.txt',index=False, sep="\t")

## Extract SH3GL2 

In [28]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

# PODXL: gene positions on hg38 (from https://useast.ensembl.org/index.html)
/home/jupyter/tools/plink2 \
--pfile chr9_FIN_release5 \
--chr 9 \
--from-bp 17579066  \
--to-bp 17797124 \
--make-bed \
--out pheno_SH3GL2_FIN

PLINK v2.00a3.3LM AVX2 Intel (3 Jun 2022)      www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_SH3GL2_FIN.log.
Options in effect:
  --chr 9
  --from-bp 17579066
  --make-bed
  --out pheno_SH3GL2_FIN
  --pfile chr9_FIN_release5
  --to-bp 17797124

Start time: Mon Jul 10 07:45:45 2023
7450 MiB RAM detected; reserving 3725 MiB for main workspace.
Using up to 2 compute threads.
16 samples (5 females, 11 males; 16 founders) loaded from
chr9_FIN_release5.psam.
80015 variants loaded from chr9_FIN_release5.pvar.
1 binary phenotype loaded (10 cases, 3 controls).
287 variants remaining after main filters.
Writing pheno_SH3GL2_FIN.fam ... done.
Writing pheno_SH3GL2_FIN.bim ... done.
Writing pheno_SH3GL2_FIN.bed ... 0%done.
End time: Mon Jul 10 07:45:45 2023


### Visualize plink files bim, fam and bed  and turn binary files to vcf

In [29]:
%%bash

# Visualize bim file
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

head pheno_SH3GL2_FIN.bim


9	chr9:17579372:T:C	0	17579372	C	T
9	chr9:17579692:T:G	0	17579692	G	T
9	chr9:17579765:C:G	0	17579765	G	C
9	chr9:17580618:A:T	0	17580618	T	A
9	chr9:17581041:G:A	0	17581041	A	G
9	chr9:17581170:C:G	0	17581170	G	C
9	chr9:17581244:C:CTA	0	17581244	CTA	C
9	chr9:17581663:G:A	0	17581663	A	G
9	chr9:17583290:G:A	0	17583290	A	G
9	chr9:17583312:A:G	0	17583312	G	A


In [30]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

head pheno_SH3GL2_FIN.fam

#Store second column of the temp.fam file in a new IDs.txt file
#awk '{print $2}' pheno_SH3GL2_FIN.fam > IDs.txt

#Count lines temp.fam file
wc -l pheno_SH3GL2_FIN.fam

0	BBDP_000507_s1	0	0	1	-9
0	COPN_000049_s1	0	0	2	2
0	CORIELL_000101_s1	0	0	2	2
0	CORIELL_000183_s1	0	0	1	2
0	CORIELL_002334_s1	0	0	1	2
0	CORIELL_002616_s1	0	0	1	2
0	CORIELL_003081_s1	0	0	1	2
0	CORIELL_003353_s1	0	0	1	2
0	CORIELL_004059_s1	0	0	1	2
0	CORIELL_004505_s1	0	0	1	1
16 pheno_SH3GL2_FIN.fam


In [31]:
SH3GL2_fam = pd.read_csv(f'{WORK_DIR}/pheno_SH3GL2_FIN.fam', sep = '\t', names=["FID", "IID", "PID", "MID","SEX","PHENO"])
SH3GL2_fam.PHENO.value_counts()

 2    10
-9     3
 1     3
Name: PHENO, dtype: int64

In [32]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

head pheno_SH3GL2_FIN.bed

l�(�)��"(�"���"����2��"���"���"�ۃ�2��"���"���"���"�ۃ�2��"�2���2��"λ�
(��
(��
8��
8��
8��
8��
(��
8��
(��


In [33]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

# Turn binary files into VCF
/home/jupyter/tools/plink \
--bfile pheno_SH3GL2_FIN \
--recode vcf-fid \
--out pheno_SH3GL2_FIN

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_SH3GL2_FIN.log.
Options in effect:
  --bfile pheno_SH3GL2_FIN
  --out pheno_SH3GL2_FIN
  --recode vcf-fid

7450 MB RAM detected; reserving 3725 MB for main workspace.
287 variants loaded from .bim file.
16 people (11 males, 5 females) loaded from .fam.
13 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 16 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%56%57%58%59%60%61%62%63%64%65%

## Annotate SH3GL2 variants

In [34]:
%%bash
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR
export PATH=$PATH:/home/jupyter/tools/rvtests/third/tabix-0.2.6/

### Bgzip and Tabix
bgzip pheno_SH3GL2_FIN.vcf

tabix -f -p vcf pheno_SH3GL2_FIN.vcf.gz
tabix -f -p vcf pheno_SH3GL2_FIN.vcf.gz

[bgzip] can't create pheno_SH3GL2_FIN.vcf.gz: File exists


In [35]:
%%bash
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

### Annotate variants using ANNOVAR: https://annovar.openbioinformatics.org/en/latest/ 
perl /home/jupyter/tools/annovar/table_annovar.pl pheno_SH3GL2_FIN.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out pheno_SH3GL2_FIN.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 pheno_SH3GL2_FIN.vcf.gz > pheno_SH3GL2_FIN.annovar.avinput>
NOTICE: Finished reading 294 lines from VCF file
NOTICE: A total of 287 locus in VCF file passed QC threshold, representing 280 SNPs (168 transitions and 112 transversions) and 7 indels/substitutions
NOTICE: Finished writing allele frequencies based on 4480 SNP genotypes (2688 transitions and 1792 transversions) and 112 indels/substitutions for 16 samples

NOTICE: Running with system command </home/jupyter/tools/annovar/table_annovar.pl pheno_SH3GL2_FIN.annovar.avinput /home/jupyter/tools/annovar/humandb/ -buildver hg38 -outfile pheno_SH3GL2_FIN.annovar -remove -protocol refGene,clinvar_20140902 -operation g,f --nopolish -nastring . -otherinfo>
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refGene

NOTICE: Running with system command <annotate_variation.pl -geneann

## Extract coding and non-syn variants

In [36]:
# Visualize multianno file
SH3GL2_FIN = pd.read_csv(f'{WORK_DIR}/pheno_SH3GL2_FIN.annovar.hg38_multianno.txt', sep = '\t')
SH3GL2_FIN.head()

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo19,Otherinfo20,Otherinfo21,Otherinfo22,Otherinfo23,Otherinfo24,Otherinfo25,Otherinfo26,Otherinfo27,Otherinfo28
0,9,17579372,17579372,T,C,intronic,SH3GL2,.,.,.,...,0/1,1/1,0/1,0/0,0/1,0/0,./.,0/1,0/1,1/1
1,9,17579692,17579692,G,T,intronic,SH3GL2,.,.,.,...,0/1,1/1,0/1,0/0,0/1,0/0,0/0,0/1,0/1,0/0
2,9,17579765,17579765,C,G,intronic,SH3GL2,.,.,.,...,0/1,1/1,0/1,0/0,0/1,0/0,0/0,0/1,0/0,0/0
3,9,17580618,17580618,A,T,intronic,SH3GL2,.,.,.,...,0/1,1/1,0/1,0/0,0/1,0/0,0/0,0/1,0/0,0/0
4,9,17581041,17581041,G,A,intronic,SH3GL2,.,.,.,...,0/1,0/0,0/1,1/1,0/0,1/1,0/1,0/1,0/0,0/0


In [37]:
SH3GL2_FIN.count()

Chr                   287
Start                 287
End                   287
Ref                   287
Alt                   287
Func.refGene          287
Gene.refGene          287
GeneDetail.refGene    287
ExonicFunc.refGene    287
AAChange.refGene      287
clinvar_20140902      287
Otherinfo1            287
Otherinfo2            287
Otherinfo3            287
Otherinfo4            287
Otherinfo5            287
Otherinfo6            287
Otherinfo7            287
Otherinfo8            287
Otherinfo9            287
Otherinfo10           287
Otherinfo11           287
Otherinfo12           287
Otherinfo13           287
Otherinfo14           287
Otherinfo15           287
Otherinfo16           287
Otherinfo17           287
Otherinfo18           287
Otherinfo19           287
Otherinfo20           287
Otherinfo21           287
Otherinfo22           287
Otherinfo23           287
Otherinfo24           287
Otherinfo25           287
Otherinfo26           287
Otherinfo27           287
Otherinfo28 

In [38]:
# Filter exonic variants
coding_FIN = SH3GL2_FIN[SH3GL2_FIN['Func.refGene'] == 'exonic']
coding_FIN.count()

Chr                   0
Start                 0
End                   0
Ref                   0
Alt                   0
Func.refGene          0
Gene.refGene          0
GeneDetail.refGene    0
ExonicFunc.refGene    0
AAChange.refGene      0
clinvar_20140902      0
Otherinfo1            0
Otherinfo2            0
Otherinfo3            0
Otherinfo4            0
Otherinfo5            0
Otherinfo6            0
Otherinfo7            0
Otherinfo8            0
Otherinfo9            0
Otherinfo10           0
Otherinfo11           0
Otherinfo12           0
Otherinfo13           0
Otherinfo14           0
Otherinfo15           0
Otherinfo16           0
Otherinfo17           0
Otherinfo18           0
Otherinfo19           0
Otherinfo20           0
Otherinfo21           0
Otherinfo22           0
Otherinfo23           0
Otherinfo24           0
Otherinfo25           0
Otherinfo26           0
Otherinfo27           0
Otherinfo28           0
dtype: int64

In [39]:
# Filter exonic and syn vars
coding_synonymous_FIN = SH3GL2_FIN[(SH3GL2_FIN['Func.refGene'] == 'exonic') & (SH3GL2_FIN['ExonicFunc.refGene'] == 'synonymous SNV')]
coding_synonymous_FIN.count()

Chr                   0
Start                 0
End                   0
Ref                   0
Alt                   0
Func.refGene          0
Gene.refGene          0
GeneDetail.refGene    0
ExonicFunc.refGene    0
AAChange.refGene      0
clinvar_20140902      0
Otherinfo1            0
Otherinfo2            0
Otherinfo3            0
Otherinfo4            0
Otherinfo5            0
Otherinfo6            0
Otherinfo7            0
Otherinfo8            0
Otherinfo9            0
Otherinfo10           0
Otherinfo11           0
Otherinfo12           0
Otherinfo13           0
Otherinfo14           0
Otherinfo15           0
Otherinfo16           0
Otherinfo17           0
Otherinfo18           0
Otherinfo19           0
Otherinfo20           0
Otherinfo21           0
Otherinfo22           0
Otherinfo23           0
Otherinfo24           0
Otherinfo25           0
Otherinfo26           0
Otherinfo27           0
Otherinfo28           0
dtype: int64

In [40]:
# Filter exonic and non-syn vars
coding_nonsynonymous_FIN = SH3GL2_FIN[(SH3GL2_FIN['Func.refGene'] == 'exonic') & (SH3GL2_FIN['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_nonsynonymous_FIN.count()

Chr                   0
Start                 0
End                   0
Ref                   0
Alt                   0
Func.refGene          0
Gene.refGene          0
GeneDetail.refGene    0
ExonicFunc.refGene    0
AAChange.refGene      0
clinvar_20140902      0
Otherinfo1            0
Otherinfo2            0
Otherinfo3            0
Otherinfo4            0
Otherinfo5            0
Otherinfo6            0
Otherinfo7            0
Otherinfo8            0
Otherinfo9            0
Otherinfo10           0
Otherinfo11           0
Otherinfo12           0
Otherinfo13           0
Otherinfo14           0
Otherinfo15           0
Otherinfo16           0
Otherinfo17           0
Otherinfo18           0
Otherinfo19           0
Otherinfo20           0
Otherinfo21           0
Otherinfo22           0
Otherinfo23           0
Otherinfo24           0
Otherinfo25           0
Otherinfo26           0
Otherinfo27           0
Otherinfo28           0
dtype: int64

In [41]:
coding_nonsynonymous_FIN.to_csv(f'{WORK_DIR}/coding_nonsynonymous.txt', sep = '\t', index = False)

In [42]:
# Filter intronic variants
intronic_FIN = SH3GL2_FIN[SH3GL2_FIN['Func.refGene'] == 'intronic']
intronic_FIN.count()

Chr                   286
Start                 286
End                   286
Ref                   286
Alt                   286
Func.refGene          286
Gene.refGene          286
GeneDetail.refGene    286
ExonicFunc.refGene    286
AAChange.refGene      286
clinvar_20140902      286
Otherinfo1            286
Otherinfo2            286
Otherinfo3            286
Otherinfo4            286
Otherinfo5            286
Otherinfo6            286
Otherinfo7            286
Otherinfo8            286
Otherinfo9            286
Otherinfo10           286
Otherinfo11           286
Otherinfo12           286
Otherinfo13           286
Otherinfo14           286
Otherinfo15           286
Otherinfo16           286
Otherinfo17           286
Otherinfo18           286
Otherinfo19           286
Otherinfo20           286
Otherinfo21           286
Otherinfo22           286
Otherinfo23           286
Otherinfo24           286
Otherinfo25           286
Otherinfo26           286
Otherinfo27           286
Otherinfo28 

In [43]:
# Filter UTR3 variants
UTR3_FIN= SH3GL2_FIN[SH3GL2_FIN['Func.refGene'] == ('UTR3')]
UTR3_FIN.count()

Chr                   1
Start                 1
End                   1
Ref                   1
Alt                   1
Func.refGene          1
Gene.refGene          1
GeneDetail.refGene    1
ExonicFunc.refGene    1
AAChange.refGene      1
clinvar_20140902      1
Otherinfo1            1
Otherinfo2            1
Otherinfo3            1
Otherinfo4            1
Otherinfo5            1
Otherinfo6            1
Otherinfo7            1
Otherinfo8            1
Otherinfo9            1
Otherinfo10           1
Otherinfo11           1
Otherinfo12           1
Otherinfo13           1
Otherinfo14           1
Otherinfo15           1
Otherinfo16           1
Otherinfo17           1
Otherinfo18           1
Otherinfo19           1
Otherinfo20           1
Otherinfo21           1
Otherinfo22           1
Otherinfo23           1
Otherinfo24           1
Otherinfo25           1
Otherinfo26           1
Otherinfo27           1
Otherinfo28           1
dtype: int64

In [44]:
# Filter UTR5 variants
UTR5_FIN = SH3GL2_FIN[SH3GL2_FIN['Func.refGene'] == ('UTR5')]
UTR5_FIN.count()

Chr                   0
Start                 0
End                   0
Ref                   0
Alt                   0
Func.refGene          0
Gene.refGene          0
GeneDetail.refGene    0
ExonicFunc.refGene    0
AAChange.refGene      0
clinvar_20140902      0
Otherinfo1            0
Otherinfo2            0
Otherinfo3            0
Otherinfo4            0
Otherinfo5            0
Otherinfo6            0
Otherinfo7            0
Otherinfo8            0
Otherinfo9            0
Otherinfo10           0
Otherinfo11           0
Otherinfo12           0
Otherinfo13           0
Otherinfo14           0
Otherinfo15           0
Otherinfo16           0
Otherinfo17           0
Otherinfo18           0
Otherinfo19           0
Otherinfo20           0
Otherinfo21           0
Otherinfo22           0
Otherinfo23           0
Otherinfo24           0
Otherinfo25           0
Otherinfo26           0
Otherinfo27           0
Otherinfo28           0
dtype: int64

In [45]:
# Calculate freq - cases vs controlS

In [46]:
reduced_coding_nonsynonymous_FIN = coding_nonsynonymous_FIN[["Chr", "Start", "End", "Gene.refGene"]]
reduced_coding_nonsynonymous_FIN.to_csv(f'{WORK_DIR}/reduced_coding_nonsynonymous_FIN.txt', sep = '\t', index = False, header= False)

In [47]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

head reduced_coding_nonsynonymous_FIN.txt

In [48]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_SH3GL2_FIN --extract range reduced_coding_nonsynonymous_FIN.txt --assoc --out coding_nonsynonymous_pheno_SH3GL2_FIN --allow-no-sex

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_SH3GL2_FIN.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile pheno_SH3GL2_FIN
  --extract range reduced_coding_nonsynonymous_FIN.txt
  --out coding_nonsynonymous_pheno_SH3GL2_FIN

7450 MB RAM detected; reserving 3725 MB for main workspace.
287 variants loaded from .bim file.
16 people (11 males, 5 females) loaded from .fam.
13 phenotype values loaded from .fam.


Error: All variants excluded by '--extract range'.


CalledProcessError: Command 'b"\nWORK_DIR='/home/jupyter/SH3GL2_GP2/'\ncd $WORK_DIR\n\n/home/jupyter/tools/plink --bfile pheno_SH3GL2_FIN --extract range reduced_coding_nonsynonymous_FIN.txt --assoc --out coding_nonsynonymous_pheno_SH3GL2_FIN --allow-no-sex\n"' returned non-zero exit status 13.

In [49]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_SH3GL2_FIN --extract range reduced_coding_nonsynonymous_FIN.txt --make-bed --max-maf 0.05 --out coding_nonsynonymous_FIN_rare_0.05 --allow-no-sex

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_FIN_rare_0.05.log.
Options in effect:
  --allow-no-sex
  --bfile pheno_SH3GL2_FIN
  --extract range reduced_coding_nonsynonymous_FIN.txt
  --make-bed
  --max-maf 0.05
  --out coding_nonsynonymous_FIN_rare_0.05

7450 MB RAM detected; reserving 3725 MB for main workspace.
287 variants loaded from .bim file.
16 people (11 males, 5 females) loaded from .fam.
13 phenotype values loaded from .fam.


Error: All variants excluded by '--extract range'.


CalledProcessError: Command 'b"\nWORK_DIR='/home/jupyter/SH3GL2_GP2/'\ncd $WORK_DIR\n\n/home/jupyter/tools/plink --bfile pheno_SH3GL2_FIN --extract range reduced_coding_nonsynonymous_FIN.txt --make-bed --max-maf 0.05 --out coding_nonsynonymous_FIN_rare_0.05 --allow-no-sex\n"' returned non-zero exit status 13.

In [50]:
SH3GL2_FIN_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_SH3GL2_FIN.assoc', delim_whitespace=True)
SH3GL2_FIN_freq.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter/SH3GL2_GP2//coding_nonsynonymous_pheno_SH3GL2_FIN.assoc'

## Calculate freq of HMZ in cases versus controls

In [51]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_SH3GL2_FIN --extract range reduced_coding_nonsynonymous_FIN.txt --recode A --out coding_nonsynonymous_pheno_SH3GL2_FIN

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_SH3GL2_FIN.log.
Options in effect:
  --bfile pheno_SH3GL2_FIN
  --extract range reduced_coding_nonsynonymous_FIN.txt
  --out coding_nonsynonymous_pheno_SH3GL2_FIN
  --recode A

7450 MB RAM detected; reserving 3725 MB for main workspace.
287 variants loaded from .bim file.
16 people (11 males, 5 females) loaded from .fam.
13 phenotype values loaded from .fam.


Error: All variants excluded by '--extract range'.


CalledProcessError: Command 'b"\nWORK_DIR='/home/jupyter/SH3GL2_GP2/'\ncd $WORK_DIR\n\n/home/jupyter/tools/plink --bfile pheno_SH3GL2_FIN --extract range reduced_coding_nonsynonymous_FIN.txt --recode A --out coding_nonsynonymous_pheno_SH3GL2_FIN\n"' returned non-zero exit status 13.

In [52]:
#Some errors may appear if no variants are found
SH3GL2_FIN_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_SH3GL2_FIN.assoc', delim_whitespace=True)
SH3GL2_FIN_freq.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter/SH3GL2_GP2//coding_nonsynonymous_pheno_SH3GL2_FIN.assoc'

In [53]:
#Some errors may appear if the pheno are not located in the previous or this script chunk
SH3GL2_FIN_recode = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_SH3GL2_FIN.raw', delim_whitespace=True)
SH3GL2_FIN_recode.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter/SH3GL2_GP2//coding_nonsynonymous_pheno_SH3GL2_FIN.raw'

In [54]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile  pheno_SH3GL2_FIN --extract range reduced_coding_nonsynonymous_FIN.txt --recode A --out coding_nonsynonymous_pheno_SH3GL2_FIN

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_SH3GL2_FIN.log.
Options in effect:
  --bfile pheno_SH3GL2_FIN
  --extract range reduced_coding_nonsynonymous_FIN.txt
  --out coding_nonsynonymous_pheno_SH3GL2_FIN
  --recode A

7450 MB RAM detected; reserving 3725 MB for main workspace.
287 variants loaded from .bim file.
16 people (11 males, 5 females) loaded from .fam.
13 phenotype values loaded from .fam.


Error: All variants excluded by '--extract range'.


CalledProcessError: Command 'b"\nWORK_DIR='/home/jupyter/SH3GL2_GP2/'\ncd $WORK_DIR\n\n/home/jupyter/tools/plink --bfile  pheno_SH3GL2_FIN --extract range reduced_coding_nonsynonymous_FIN.txt --recode A --out coding_nonsynonymous_pheno_SH3GL2_FIN\n"' returned non-zero exit status 13.

In [55]:
# Explore HMZs for chr9: in cases
SH3GL2_hom_cases_FIN = SH3GL2_FIN_recode[(SH3GL2_FIN_recode['chr9:'] == 2) & (SH3GL2_FIN_recode['PHENOTYPE'] == 2)]
SH3GL2_hom_cases_FIN.head()

NameError: name 'SH3GL2_FIN_recode' is not defined

In [56]:
SH3GL2_hom_cases_FIN.shape

NameError: name 'SH3GL2_hom_cases_FIN' is not defined

In [57]:
# Explore HMZs for chr9: in controls
SH3GL2_hom_controls_FIN = SH3GL2_FIN_recode[(SH3GL2_FIN_recode['chr9:'] == 2) & (SH3GL2_FIN_recode['PHENOTYPE'] == 1)]
SH3GL2_hom_controls_FIN.head()

NameError: name 'SH3GL2_FIN_recode' is not defined

In [58]:
SH3GL2_hom_controls_FIN.shape

NameError: name 'SH3GL2_hom_controls_FIN' is not defined

## Save out results..!

In [59]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR} {WORKSPACE_BUCKET}')

Executing: gsutil -mu terra-18d8e41c cp -r /home/jupyter/SH3GL2_GP2/ gs://fc-f7a400c1-827e-48f8-b7b6-90c488a000a4


Copying file:///home/jupyter/SH3GL2_GP2/pheno_SH3GL2_EUR.annovar.avinput [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/SH3GL2_GP2/chr9_AJ_release5.pgen [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/SH3GL2_GP2/pheno_SH3GL2_AJ.annovar.hg38_multianno.vcf [Content-Type=text/vcard]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of c